In [48]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


**Aim:** Show competence in using the `pandas` library and knowledge of the correlation concept.

In the following you are asked to analyze a real world dataset: the Exasens Data Set from https://archive.ics.uci.edu/ml/datasets/Exasens which contains missing values.

In this exercise data matrices are considered row-wise, that is a data matrix for `m` instances is a `m x n` matrix consisting of `m` vectors in `n` dimensions.

## Question 1.1

Make a function `load_exasens_data_set()` that returns a pandas data frame with the the data made available by the UC Irvine Machine Learning Repository at the following URL: https://archive.ics.uci.edu/ml/datasets/Exasens (you need to find the exact URL for the data file in csv format).


The function should read the data directly from the URL (not froma location on the local disk) and  perform the necessary processing and manipulation of the raw data so as to output a data frame `df` with the following characteristics:
```
>>df.info()

RangeIndex: 399 entries, 0 to 398
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Diagnosis           399 non-null    object 
 1   ID                  399 non-null    object 
 2   Imaginary_Part_Min  100 non-null    float64
 3   Imaginary_Part_Avg  100 non-null    float64
 4   Real_Part_Min       100 non-null    float64
 5   Real_Part_Avg       100 non-null    float64
 6   Gender              399 non-null    int64  
 7   Age                 399 non-null    int64  
 8   Smoking             399 non-null    int64  
dtypes: float64(4), int64(3), object(2)
memory usage: 28.2+ KB
```
**Note:** you have to obtain a frame that has the **exact** column names as shown above.

In [49]:
def load_exasens_data_set():
    df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00523/Exasens.csv')
    df = df[2:]
    df.drop(list(df.columns)[9:13], axis=1, inplace=True)
    df.rename(columns={"Imaginary Part": "Imaginary_Part_Min", "Unnamed: 3": "Imaginary_Part_Avg","Real Part": "Real_Part_Min", "Unnamed: 5": "Real_Part_Avg"},inplace=True)
    df = df.astype({'Imaginary_Part_Min':'float64','Imaginary_Part_Avg':'float64','Real_Part_Min':'float64',"Real_Part_Avg":'float64','Gender':'int64','Age':'int64','Smoking':'int64'})
    df.index = range(len(df.index))
    return df


In [50]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 1.2

Make a function `add_binned_column(df, col_name, limits)`. The function takes in input a data frame `df`, one of its columns label `col_name` and a list of values `limits`. 

The function returns the input data frame with an additional column: the label of the new column is obtained by adding the fixed suffix "_bin", so if col_name is "Age" then the new column has label "Age_bin". 

The aim of the function is to convert values into the ordinal integers 0,1,2,...,etc. 

The list `limits` contains the bins boundaries used for the conversion. The bins are closed on the left and open on the right, i.e. a bin with boundaries `[10,20)` contains all values >= 10 and < 20. 

In a list `[v, a, b, c, ..., w]` the first value `v` defines the bin (-inf,v) and the last value `w` defines the bin (w,+inf).

For example `limits=[5,10,15]` defines the following bins: `(-inf,5), [5,10), [10, 15), [15, +inf)`. Values falling in the first bin will be mapped to the integer 0, values in the second bin will be mapped to the integer 1,  in the third bin to 2, in the fourth bin to 3. 


In [51]:
def add_binned_column(df, col_name, limits):
    if not float('-inf') in limits:
        limits.append(float('-inf'))
    if not float('inf') in limits:
        limits.append(float('inf'))
    limits = sorted(limits)
    df[col_name+'_bin'] = pd.cut(df[col_name],bins=limits,labels=range(len(limits)-1))
    return df


In [52]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 1.3

Make a function `add_class(df, col_name)` to return a data frame with an added new column with fixed label of "target".

The function considers the distinct categorical values present in the column with label `col_name` in the input data frame `df` and maps them to integers.

For example, if the data frame `df` has a column with label `Diagnosis` with values `[COPD, COPD, HC, Asthma, HC, COPD]`, then the new column `target` will have values `[0,0,1,2,1,0]`.

In [53]:
def add_class(df, col_name):
    df['target']=df[col_name].factorize()[0]
    return df


In [54]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 1.4

Make a function `replace_missing_with_conditional_mean(df, condition_cols, cols)` to replace the missing values present in columns with labels in the list `cols`. 

The value to be replaced is computed as the mean of the non missing values of the corresponding group. Groups are formed based on the values in the columns with labels in the list `condition_cols`. 


For example if you have the following data frame `df`:
```
  Col1 Col2  Col3
0    A    c   1.0
1    A    c   3.0
2    B    c   5.0
3    A    d   6.0
4    A    c   NaN
```
then applying `replace_missing_with_conditional_mean(df, condition_cols=['Col1','Col2'], cols=['Col3'])` should yield in output
```
  Col1 Col2  Col3
0    A    c   1.0
1    A    c   3.0
2    B    c   5.0
3    A    d   6.0
4    A    c   2.0
```
this is because the record on line 4 belongs to the group `A c` that has a mean of `(1+3)/2 = 2`.

In [55]:
def replace_missing_with_conditional_mean(df, condition_cols, cols):
    for col in cols:
        df[col]=df.groupby(condition_cols)[col].apply(lambda column: column.fillna(column.mean()))
    return df


In [56]:
# This cell is reserved for the unit tests. Do not consider this cell. 

### Question 1.5

Make a function `standardize(df, cols)` that outputs a data frame where the values for the columns with labels in the list `cols` have been standardized, that is, they have been processed to have zero mean and standard deviation of 1.

In [57]:
def standardize(df, cols):
    new_df = df[cols]
    new_df = (new_df - new_df.mean()) / new_df.std()
    df[cols] = new_df[cols]
    return df


In [58]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 1.6

Make a function `make_correlation_matrix_from(df, cols)` to output a numpy array containing the correlation matrix relative to the data in the input data frame `df`. The data is extracted from the columns with labels in `cols`.

Provide your own implementation of the correlation. Do **not** use functions from the numpy library or any other library to compute directly the correlation matrix.

In [59]:
def make_correlation_matrix_from(df,cols):
    corr=np.zeros(shape=(len(cols),len(cols))) # empty array
    i=0;j=0
    n=df.shape[0]
    for a in range(len(cols)):
        for b in range(len(cols)):
            x=np.nan_to_num(df[cols[a]].values)
            y=np.nan_to_num(df[cols[b]].values)
            cor=(sum((x-x.mean())*(y-y.mean())))/(n)/(x.std()*y.std()) 
            corr[a,b]=cor
    return corr

In [60]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 1.7

Make a function `plot_corr(corr, labels)` to plot a correlation matrix `corr` (with variable names given as a list of strings in `labels`) (2 marks).


When you execute the following sequence of tranformations
```python
df = load_exasens_data_set()
df = add_binned_column(df, col_name='Age', limits=[30,50,60])
df = add_class(df, col_name='Diagnosis')
df = replace_missing_with_conditional_mean(df, condition_cols=['Smoking','Gender','Age_bin'], cols=['Imaginary_Part_Min','Imaginary_Part_Avg','Real_Part_Min', 'Real_Part_Avg'])
df = standardize(df, cols=['Imaginary_Part_Min','Imaginary_Part_Avg','Real_Part_Min', 'Real_Part_Avg','Gender','Age','Smoking'])
corr =  make_correlation_matrix_from(df, cols=['Imaginary_Part_Min','Imaginary_Part_Avg','Real_Part_Min', 'Real_Part_Avg','Gender','Age','Smoking'])
plot_corr(corr, labels=cols)
```
you should obtain (3 marks):
<img src='corr.png' width="500">

In [61]:
import seaborn as sns 
def plot_corr(C, labels):
    sns.heatmap(C, annot = True,cmap='PuBu',xticklabels=labels,yticklabels=labels,cbar=False,fmt='0.2g')
    plt.show()


In [62]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [63]:
# This cell is reserved for the unit tests. Do not consider this cell. 

# Part 2

**Aim:** Show an understanding of the main concepts in linear algebra such as PCA and SVD.


In this exercise data matrices are considered row-wise, that is a data matrix for `m` instances is a `m x n` matrix consisting of `m` vectors in `n` dimensions.

## Question 2.1

Write the function `get_mean_cov(X)` that takes a data matrix in input and returns the mean as a one dimensional numpy vector and the covariance matrix as a numpy matrix object.


Provide your own implementation of the covariance. Do **not** use functions from the numpy library or any other library to compute directly the covariance matrix.

In [64]:
def get_mean_cov(X):
    mean = np.mean(X, axis=0)
    Xmat = np.mat(X - mean.reshape(1,-1))
    cov = Xmat.T*Xmat/(Xmat.shape[0]-1)
    return np.array(mean).ravel(), cov
    
def getmean(X):
    return np.array([sum(X[row])/len(X[row]) for row in range(len(X))])


In [65]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2.2

Write your own code to perform the PCA dimensionality reduction (i.e. do not use functions provided by the `scikit` library, such as `sklearn.decomposition.PCA` or `sklearn.decomposition.TruncatedSVD` or any other library).

Write a function `PCA(X, n_dim=2)` that takes in input a `m x n` data matrix consisting of `m` vectors in `n` dimensions and returns the **centred** projection of `X` on the `n_dim` principal components as a `m x n_dim` data matrix.

In [66]:
def PCA(X,n_dim=2):
    mean,cov=get_mean_cov(X)
    X = np.array(X)
    X = X - X.mean(axis=0)
    v, u = np.linalg.eig(cov)
    idx = v.argsort()[::-1] 
    v = v[idx] 
    u = u[:,idx] #
    return X.dot(u[:, :n_dim])


In [67]:
# This cell is reserved for the unit tests. Do not consider this cell. 

In [68]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2.3

Write a function `plot(X, y=None)` that exploits the PCA function to plot a two dimensional projection of the input `m x n` data matrix `X` and that,  if available, uses the array `y` to determine the colors. 

In [69]:
def plot(X,y=None):
    X2d = PCA(X, 2)
    plt.figure(figsize=(8,8))
    a,b=X2d.A.T
    if y is None:
        plt.scatter(a,b)
    else:
        plt.scatter(a,b, c=y)
    plt.axis('equal')
    plt.grid()
    plt.show()


In [70]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2.4

Write a function `approximate(X, r)` that returns the rank `r` approximation of the input data matrix `X`, i.e. for a `m x n` input matrix `X` returns a `m x n` matrix `Z` that has rank `r`.

In [71]:
def approximate(X, r):
    Xc = np.where(np.isnan(X), np.ma.array(X, mask = np.isnan(X)).mean(axis = 0), X)
    U, s, Vh = np.linalg.svd(np.mat(Xc))
    S = np.mat(np.diag(s[:r]))
    return U[:,:r] * S * Vh[:r, :]


In [72]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2.5

Write a function `approximation_error(X, r)` that returns an array with the lenghts of the difference vector between the instances in the input data matrix `X` and the corresponding approximated matrix of rank `r`. When `X` is a `m x n` data matrix, the output is a one dimensional array of size `m`.

In [73]:
def approximation_error(X,r):
    Xd = X-approximate(X,r)
    return np.sqrt(np.sum(np.power(Xd,2),axis=0))

In [74]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2.6

Write a function `replace_missing_with_approximation(X, is_missing, r)` that receives in input a data matrix `X` and a boolean array `is_missing` and a desired rank `r`. The function computes a rank `r` approximation of `X` and replaces the vectors corrisponding the position containing `True` in the boolean array `is_missing`. The output data matrix has the same shape of the input data matrix.

In [75]:
def replace_missing_with_approximation(X, is_missing, r):
    X = X.astype(float)
    Xr = approximate(X,r)
    X[is_missing] = Xr[is_missing]
    return X

In [76]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2.7


Write a function `remove_outliers(X, y, threshold)` that receives in input a data matrix `X` and a class vector `y` and outputs a data matrix `Xp` and a class vector `yp`. Let `Xi` be the data matrix relative to class `i`. The function computes the mahalanobis distance of each vector in `Xi` with respect to the distribution that can be inferred from `Xi`. Only the vectors with a distance inferior than the desired parameter `threshold` are reported in output, i.e. this function removes the instances that have a large mahalanobis distance. 

Note that the order of the vector in `X` is not necessarily related to the order of the vectors in `Xp`. 
It is understood that the entries in `Xp` and `yp` are aligned (i.e. the class of the row vector with row index `j` is `yp[j]`).

In [77]:
def remove_outliers(X,y,threshold):
    xp=np.array([]).reshape(0,X.shape[1])
    yp=np.array([])
    for i in np.unique(y):
        x=X[y==i]
        y1=y[y==i]
        x_minus_mu = x - np.mean(X,axis=0)
        cov = np.cov(x.T)
        left_term = np.dot(x_minus_mu, np.linalg.inv(cov))
        mahal = np.dot(left_term, x_minus_mu.T)
        m=mahal.diagonal()
        ind=m<threshold
        x1=x[ind]
        y1=y1[ind]
        xp=np.append(xp,x1,axis=0)
        yp=np.append(yp,y1,axis=0)
    return xp,yp


In [78]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 2.8

The functions created are now going to be applied on the exasens dataset (i.e. you do not need to write anything for this part, it will be run automatically). This is to test that you have succesfully produced all the necessary functions. 

In the next cell the following test is going to be performed:

```python
df = load_exasens_data_set()
df = add_binned_column(df, col_name='Age', limits=[30,50,60])
df = add_class(df, col_name='Diagnosis')
df = replace_missing_with_conditional_mean(df, condition_cols=['Smoking','Gender','Age_bin'], cols=['Imaginary_Part_Min','Imaginary_Part_Avg','Real_Part_Min', 'Real_Part_Avg'])

# extract the data matrix: X, the class vector: y and the missing elements array: is_missing
# ...

Xp = replace_missing_with_approximation(X, is_missing, r=2)
Xc,yc = remove_outliers(Xp,y, threshold=5)
assert Xc.shape[0] < (?), 'The number of instances selected as non outliers should be less than (?)'

```

In [79]:
# This cell is reserved for the unit tests. Do not consider this cell. 

# Part 3


**Aim:** demonstrate an understanding of LDA. 

In this exercise you will need to build a dataset using the `multivariate_normal` function provided by the numpy library. You will then fit a multiclass LDA model to your data and finally you will display the decision boundaries of the trained predictive model and measure the accuracy of the resulting classifier. 


**Construction of the dataset:**

Given a parameter `k`, the dataset is generated using `k` multi variate normal data generators. All instances are 2 dimensional. 


At the end for `make_data(k=5, num_instances=1000, radius=10, ratio=2, rotation=0)` you should obtain something like this:


<center><img src="dat.png" alt="ex" width="400"/></center>


**LDA classifier:**

At the end your implementation of the LDA classifier should allow you to obtain something like this:


<center><img src="lda.png" alt="ex" width="400"/></center>


In this exercise data matrices are considered row-wise, that is a data matrix for `m` instances is a `m x n` matrix consisting of `m` vectors in `n` dimensions.

## Question 3.1

Write a function `make_means(k, radius)` that outputs a data matrix containing the 2D coordinates of `k` means.  The means of the data generator must lie on the vertices of a regular polygon (if `k=3` the polygon is a triangle, if `k=6` it is an hexagon, etc). Write your own code to determine the position of the vertices of a regular polygon given a radius value in input.

For example `make_means(3, radius=1)` would yield:
```
[[ 1. ,  0.       ],
[-0.5 ,  0.8660254],
[-0.5 , -0.8660254]]
```

 *Hint: you can use your knowledge on linear transformations (e.g. rotations).*  

In [80]:
def make_means(k, radius):
    means_mat = np.mat(np.zeros(k*2).reshape(k,2))
    A = rotate((360/k))
    means_mat[0,0] = radius
    for i in range(1,k):
        means_mat[i] = means_mat[i-1] * A
    return means_mat
    
def rotate(deg):
    theta = deg * np.pi/180.0
    A = [[np.cos(theta), -np.sin(theta)],[np.sin(theta), np.cos(theta)]]
    return np.mat(A)


In [81]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3.2

Write a function `make_covariance(ratio, rotation)` to build a covariance matrix.
The covariance matrix is constructed by specifying 2 parameters: a `ratio` between the two main directions of variability (if the ratio is, say, 2:1, then the parameter is 2); and a `rotation` in degrees (i.e. 90 for a right angle) to determine the main *direction* of variability. 

*Example:* If the ratio is 2 and the rotation is 0 your covariance matrix will be:
```[[4., 0.],
 [0., 1.]]```

*Example:* If the ratio is 2 and the rotation is 45 your covariance matrix will be:

```[[2.5, 1.5],
 [1.5, 2.5]]```

In [82]:
def make_covariance(ratio, rotation):
    R = rotate(rotation)
    S = np.mat([[ratio,0],[0,1]])
    T = R * S
    return T * T.T

In [83]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3.3

Write a function `make_data(k, num_instances, radius, ratio, rotation)` to output a data matrix and a one dimensional class vector.

Using the previous functions generate a data matrix with `num_instances` rows and 2 columns and a `targets` vector of length `num_instances` containing a class indicator for each instance (i.e. an integer between 0 and k-1). 

The function takes in input the number of classes `k`, the total number of instances `num_instances`, the parameter `radius` to express the distance from the origin for the means of the multi variate normal data generators, and finally the `ratio` and the `rotation` as specified previously for the `make_covariance` function.

In [84]:
def make_data(k, num_instances, radius, ratio, rotation):
    mu = make_means(k,radius)
    Ds = [np.random.multivariate_normal(mean=mu[i].flat,cov=make_covariance(ratio, rotation), size=num_instances) for i in range(k)]
    X = np.vstack(Ds)
    y = np.hstack([[i]*D.shape[0] for i, D in enumerate(Ds)])
    return np.mat(X),y.reshape(-1)


In [85]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3.4

Write the function `fit_LDA(X, y)`. The function outputs a the parameters for a LDA classifier fit for the classification of the required number of classes. The number of classes will be automatically deduced from the `y` class vector.


Write the function `test_LDA(data_matrix, params)` to output the class predicted by the LDA model encoded in `param`. 

You must write your own implementation of LDA. Do not use the implementation in the library `scikit` or `numpy` or any other library. 

**Note:** the classification task here is multi-class *not* binary.

In [86]:
def fit_LDA(X, y):
    classesLen = len(np.unique(y))
    pi=[] 
    mu = [] 
    COV = []
    for i in range(classesLen):
        D = X[y==i]
        p,m,cov = LDA_factors(D,X.shape[0])
        pi.append(p)
        mu.append(m)
        COV.append(cov)
    C=[]
    W=[]
    for i in range(classesLen-1): 
        for j in range(i+1,classesLen):
            S = np.mat((COV[i]+COV[j])/2)
            SI = S.I
            w = SI*(mu[i]-mu[j])
            c = np.log(pi[i]/pi[j]) -0.5 *mu[i].T * SI * mu[i] +0.5 *mu[j].T * SI * mu[j]
            W.append(w)
            C.append(c)
    return W,C,classesLen
    
def test_LDA(X, *params):
    W =params[0][0]
    C =params[0][1]
    classLen =params[0][2]
    votes = np.zeros(len(W)*len(X)).reshape(len(W),len(X))
    k=0
    for i in range(classLen-1): 
        for j in range(i+1,classLen):
            result = (X * W[k] > -C[k]).astype(int).A.reshape(-1)
            votes[k,result==1] = int(i)
            votes[k,result==0] = int(j)
            k += 1
    votes = votes.astype(int)
    return [np.argmax(np.bincount(votes[:,i])) for i in range(votes.shape[1])]

      
def LDA_factors(D,N):
    pi = D.shape[0]/N
    mu = np.mat(np.mean(D,axis=0)).reshape(-1,1)
    C = np.mat(D-np.mean(D,axis=0))
    S = C.T*C/(D.shape[0]-1)
    return pi,mu,S



In [87]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3.5

Write a function `make_grid(D, n)` to generate instances regularly spaced in 2D. The function takes in input a data matrix `D` to extract the limits, and a parameter `n` to specify the number of points per side, e.g. with `n=10` a grid of 100 points is generated. 

In [88]:
def make_grid(D, n=10):
    mn = np.min(D.A,axis=0).reshape(-1)
    mx = np.max(D.A,axis=0).reshape(-1)
    dat1 = np.linspace(mn[0],mx[0],n)
    dat2 = np.linspace(mn[1],mx[1],n)
    grid = [(x1,x2) for x1 in dat1  for x2 in dat2]
    grid = np.mat(grid)
    return grid

In [89]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3.6

Write a function `accuracy(y_true, y_pred)` to compute the fraction of correct predictions over the total number of predictions.

In [90]:
def accuracy(y_true, y_pred):
    return np.sum(y_true==y_pred)/len(y_pred)

In [91]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3.7

Write a function `plot_data_and_grid(X,y,G,preds)` to visualize the results, where `X` is the data matrix, `y` is the class vector, `G` is a data matrix containing the grid vectors and `preds` is a vector containing the class predictions for the grid data points.  

In [92]:
def plot_data_and_grid(X,y,G,preds):
    X2 = np.vstack([X,G])
    y2 = np.hstack([y,preds])
    plot(X2,y2)


In [93]:
# This cell is reserved for the unit tests. Do not consider this cell. 

## Question 3.8

The functions created are now going to be applied on the artificial dataset (i.e. you do not need to write anything for this part, it will be run automatically). This is to test that you have succesfully produced all the necessary functions. 

In the next cell the following test is going to be performed:

```python
D,targets = make_data(k=?, num_instances=?, radius=?, ratio=?, rotation=?)
G = make_grid(D,n=60)
LDA_params = fit_LDA(D,targets)
preds = test_LDA(G, LDA_params)
assert "... the accuracy should be (?) ..."

```

In [94]:
# This cell is reserved for the unit tests. Do not consider this cell. 